In [ ]:
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pyplot as plt

To make the forumlas more compact we intoduce the following shorthand

$m$: Input dim 

$p$: Output dim

$d$: State dim

Without additions this gives the cost for one stage
$$ C_k = d_{k+1}d_k + d_{k+1}m_k+p_kd_k+p_km_k $$

Including additions we get the costfor one stage

$$ C_k' = d_{k+1}(2d_k-1) + d_{k+1}(2m_k-1)+p_k(2d_k-1)+p_k(2m_k-1) $$

The overall cost of the system is the sum of the individual costs

In [ ]:
dk, dk1, dk2, mk, mk1, pk, pk1 =sp.symbols('d_k d_{k+1} d_{k+2} m_k m_{k+1} p_k p_{k+1}')

In [ ]:
cost_stage = dk1*dk +dk1*mk +pk*dk +pk*mk
cost_stage

In [ ]:
cost_stage1 = dk2*dk1 + dk2*mk1 +pk1*dk1 +pk1*mk1

cost_both = cost_stage+cost_stage1
cost_both

# Change the bounds

move the input by $\Delta_m$, the output by $\Delta_p$ and the state by $\Delta_n$

In [ ]:
Delta_m,Delta_p,Delta_d =sp.symbols('Delta_m Delta_p Delta_n')

In [ ]:
(cost_both-cost_both.subs({dk1:dk1+Delta_d,mk:mk+Delta_m,mk1:mk1-Delta_m,pk:pk+Delta_p,pk1:pk1-Delta_p})).simplify()

Lets try to get some insights:

We star with:

$$ \displaystyle - \Delta_{m} \Delta_{n} - 2 \Delta_{m} \Delta_{p} - \Delta_{m} n_{k+1} + \Delta_{m} n_{k+2} - \Delta_{m} p_{k} + \Delta_{m} p_{k+1} + \Delta_{n} \Delta_{p} - \Delta_{n} m_{k} - \Delta_{n} n_{k} - \Delta_{n} n_{k+2} - \Delta_{n} p_{k+1} - \Delta_{p} m_{k} + \Delta_{p} m_{k+1} - \Delta_{p} n_{k} + \Delta_{p} n_{k+1} $$


First we suppose that all products of $\Delta$s are close to 0 and can be ignored


$$ \displaystyle  - \Delta_{m} n_{k+1} + \Delta_{m} n_{k+2} - \Delta_{m} p_{k} + \Delta_{m} p_{k+1}  - \Delta_{n} m_{k} - \Delta_{n} n_{k} - \Delta_{n} n_{k+2} - \Delta_{n} p_{k+1} - \Delta_{p} m_{k} + \Delta_{p} m_{k+1} - \Delta_{p} n_{k} + \Delta_{p} n_{k+1} $$

we can reproup the terms to:
$$
\Delta_{m} (-n_{k+1} +  n_{k+2} - p_{k} + p_{k+1})
+  
\Delta_{p}( -m_{k} + m_{k+1} - n_{k} + n_{k+1})
+ \Delta_{n} (-m_{k} -n_{k} -n_{k+2}) 
$$

If we suppose that the two conseqcutive states are similar in the input and output dims as well as the state dims we have $m_{k} - m_{k+1} \approx 0$,$p_{k} - p_{k+1} \approx 0$ and $-n_{k+1} +  n_{k+2} \approx 0$

This leves us with the expression
$$\Delta_{n} (-m_{k} -n_{k} -n_{k+2}) $$
This motivates why an reduction of the state dimention also reduces the cost.

# Move left

Change input

$$\tilde{p}_k =p_k+1 $$
$$\tilde{p}_{k+1}= p_{k+1} -1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1})-cost_both)


Cost if $n_{k+1}$ gets samller
$$\tilde{n}_{k+1} = _{k+1}-1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1,dk1:dk1-1})-cost_both)


# Move rigth

Change input

$$\tilde{p}_k =p_k-1 $$
$$\tilde{p}_{k+1}= p_{k+1} +1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk-1,pk1:pk1+1})-cost_both)


Cost if $n_{k+1}$ gets bigger
$$\tilde{n}_{k+1} = _{k+1}+1$$

In [ ]:
sp.simplify(cost_both.subs({pk:pk+1,pk1:pk1-1,dk1:dk1+1})-cost_both)


# Move down

Change output

$$\tilde{m}_k =m_k+1 $$
$$\tilde{m}_{k+1}= m_{k+1} -1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk+1,mk1:mk1-1})-cost_both)


Cost if $n_{k+1}$ gets smaller
$$\tilde{n}_{k+1} = _{k+1}-1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk+1,mk1:mk1-1,dk1:dk1-1})-cost_both)


# Move up

Change output

$$\tilde{m}_k =m_k-1 $$
$$\tilde{m}_{k+1}= m_{k+1} +1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk-1,mk1:mk1+1})-cost_both)


Cost if $n_{k+1}$ gets bigger
$$\tilde{n}_{k+1} = _{k+1}+1$$

In [ ]:
sp.simplify(cost_both.subs({mk:mk-1,mk1:mk1+1,dk1:dk1+1})-cost_both)


# Some helping function

Transform the sum into a polynomial

calcaultes $$\sum^N_{n=1} exp(n) $$  for $$exp = \sum_{p=0}^5 a_p n^p$$

In [ ]:
def transfrom_sum(arg,n,N,n_start=None):
    #Take a sum consisting of polynomilas in n and calcualtes the overall polynomial using the Faulhabersche_Formel
    #Knuth https://arxiv.org/abs/math/9207222
    #calcaultes $$\sum^N_{n=1} exp$$  for $$exp = \sum_{p=0}^5 a_p n^p$$
    #Parameters:
    #    arg:   expression in the sum
    #    n:     running variable
    #    N:     upper end of sum
    #returns:
    # expression 
    arg = arg.expand().collect(n)
    expression = 0
    for i in range(6):
        #faulhaber coeffs form: https://de.wikipedia.org/wiki/Faulhabersche_Formel 
        #look below for more details
        coeffs = {0:N,
              1:1/sp.S(2)*N**2+1/sp.S(2)*N,
              2:1/sp.S(3)*N**3+1/sp.S(2)*N**2+1/sp.S(6)*N,
              3:1/sp.S(4)*N**4+1/sp.S(2)*N**3+1/sp.S(4)*N**2,
              4:1/sp.S(5)*N**5+1/sp.S(2)*N**4+1/sp.S(3)*N**3-1/sp.S(30)*N,
              5:1/sp.S(6)*N**6+1/sp.S(2)*N**5+5/sp.S(12)*N**4-1/sp.S(12)*N**2}


        expression = expression + (arg.coeff(n,i))*coeffs[i] 
        if n_start:
            expression = expression - (arg.coeff(n,i))*coeffs[i].subs({N:n_start-1})
            #display((arg.coeff(n,i))*coeffs[i].subs({N:n_start-1}))
    #test for higher orders
    for i in range(6,100):
        if arg.coeff(n,i) !=0:
            raise NotImplementedError("n^"+str(i)+" is not implemented")
    return expression

# Approximations of Bounds

$$ m = M/K $$
$$ m = M/K $$

To approxiamte the rank we use an approxiamtion based on the shape of the Hankel matrices.


## Shapes of the Hankel matrix

The height of $H_k$ can be computed accoring to 
$$ \Lambda_k = P -\sum_{i=1}^{k-1} p_{i}= P - (k-1)p = (K-k+1)p$$
The width of $H_k$ is given by
$$ \Upsilon_k = \sum_{i=1}^{i-1} m_{i} = (k-1)m$$

## Approximate rank

At this stage it is not clear how to approximate the number of stages, as this dpends on the distribution of the singular values as well as the required precission.
Therfore I decide to use approxiamtions that can be motivated using properties of the Hankel matrices, but are still easy enough to make it possible to give closed form approxiamtions.
This also means that these approxiamtiosn have hyperparameters that have to be adjusted.

### Minimum of height and width (here $p=m$)
The first approximation is based on the biggest possible Rank of the $H_k$ this the minimum of the width and height.
For an quadratic natrix we get
$$ \min((K-k+1)p,(k-1)m) = \min\big((v-k+1),(k-1)\big)m = \min\big(v+1-k,k-1\big)m$$

change this 
$K+1-k>k-1 \Leftrightarrow K+1>2k-1 \Leftrightarrow K+2>2k \Leftrightarrow K/2+1>k$

$$d_k = \gamma \min((K-k+1)m,(k-1)m) = \begin{cases}
    \gamma (k-1)m &\text{if } k\leq K/2+1 \\
    \gamma (K+1-k)m &\text{if } k > K/2+1 \\
\end{cases}$$


For an nonsquare matrix we get 
$$\min((K-k+1)p,(k-1)m)$$

$$
(K-k+1)p<(k-1)m \Leftrightarrow
-kp+(K+1)p<km-m \Leftrightarrow
m+(K+1)p<km+kp \Leftrightarrow
m+(K+1)p<(m+p)k \Leftrightarrow
\frac{m+(K+1)p}{(m+p)} < k
$$


$$
d_k = \gamma \min((K-k+1)\frac{P}{K},(k-1)\frac{M}{K}) = \begin{cases}
    \gamma (k-1)\frac{M}{K} &\text{if } k\leq \frac{M+(K+1)P}{(M+P)} \\
    \gamma (K+1-k)\frac{P}{K} &\text{if } k > \frac{M+(K+1)P}{(M+P)} \\
\end{cases}
$$



$$rank(H) \propto \gamma \cdot \text{height} \cdot \text{width}$$

### Product of height and width:
The second approxiamtion is based on the idea that the singular vlaues increase are related to the Frobenius norm by $\|H\|_F = \sqrt{\sum \sigma_i^2}$.
As the Frobenius norm increases, this should also result in increasing singular values.
If we consider the elements of $H$ as cosntant, then $\|H\|_F \propto \text{size(H)}$

$$(K-k+1)(k-1) \frac{PM}{K^2\min(P,M)}$$


In [ ]:
# numerical test for 

def plot_approx(K = 100,M = 500,P = 500):
    ks = np.arange(1,K+1)
    d_prod = (K-ks+1)*(ks-1)*P*M/K**2/max(M,P) #*max(M,P) #The M is for normalization
    d_min = np.min(np.vstack(((K-ks+1)*P/K,(ks-1)*M/K)),axis=0)


    plt.plot(ks,d_prod,label="prod")
    plt.plot(ks,d_min ,label="min")
    
plt.figure(figsize=[7,3])
plt.subplot(1,3,1)
plot_approx()
plt.legend()

plt.subplot(1,3,2)
plot_approx(M=250,P=500)

plt.subplot(1,3,3)
plot_approx(M=500,P=250)

In [ ]:
K,m,p,M,P = sp.symbols('K m p M P',positive=True)
N = sp.symbols('N',positive=True) #for square matrix
dk,dk1 = sp.symbols('d_k d_{k+1}',positive=True)
gamma = sp.symbols('gamma',positive=True)

If one supposes that the state dims for the casual and anticasual system are equivalent, then only the  caclulateion for the casual system is nedded.
All term except the term for the $D$-matrix are doubled.
This resualts in the cost

In [ ]:
k = sp.symbols('k')
#include D for anticausal part
#cost =nk1*nk+nk1*m+p*nk+p*m
# only include D once -> divide it by 2
cost =2*dk1*dk+2*dk1*M/K+2*P/K*dk+P*M/K**2
cost_sum = sp.Sum(cost,(k,1,K))
cost_sum

# Product

In [ ]:
d_prod = gamma*(K-k+1)*(k-1)*P*M/(K**2*N).subs(N,M) #here N is min(M,P)
d_prod_k = d_prod
d_prod_k1 = d_prod.subs(k,k+1)

cost_prod=cost_sum.subs({dk:d_prod_k,dk1:d_prod_k1})
sp.simplify(cost_prod)

In [ ]:
arg_prod = cost.subs({dk:d_prod_k,dk1:d_prod_k1})
arg_prod

In [ ]:
cost_total_prod = transfrom_sum(arg_prod,k,K)

In [ ]:
cost_total_prod.expand().collect(K)

relation between $m$ and $p$ and $K$

Here a quadratic matrix:
$$p = N/K$$
$$m = N/K$$

In [ ]:
cost_total_prod=cost_total_prod.subs({M:N,P:N}).expand().collect(K)
cost_total_prod

In [ ]:
#critical_points=sp.solve((deriv*v**4).subs({Nu:100}).expand().collect(K),K)

In [ ]:
f = sp.lambdify([K,(N,gamma)],cost_total_prod)
Ks = 2**np.arange(11)
Nn = Ks[-1]
for gamma_tilde in np.logspace(-2,0,6):
    costs= f(Ks,(1024,gamma_tilde))
    plt.semilogy(np.arange(11),costs,label="g="+str(gamma_tilde))
    i = np.argmin(costs)
    plt.scatter(i,costs[i])
    
plt.legend()
plt.grid()
plt.hlines(Nn**2,0,10)
locs, labels = plt.xticks()
plt.xticks(locs,["$2^{"+str(int(loc))+"}$" for loc in locs])
plt.xlim([0,10])

In [ ]:
cost_total_prod.subs(K,2)

## Now minimum

Here I consider only square systems. A similar derivation is also possible for nonsquare martrices but result in a slightly more difficult cost
$$d_k = \gamma \min((K-k+1)p,(k-1)m) = \begin{cases}
    \gamma (k-1)\frac{M}{K} &\text{if } k\leq K/2+1 \\
    \gamma (K+1-k)\frac{P}{K} &\text{if } k > K/2+1 \\
\end{cases}$$

$$
d_k = \gamma \min((K-k+1)\frac{P}{K},(k-1)\frac{M}{K}) = \begin{cases}
    \gamma (k-1)\frac{M}{K} &\text{if } k\leq \frac{M+(K+1)P}{(M+P)} \\
    \gamma (K+1-k)\frac{P}{K} &\text{if } k > \frac{M+(K+1)P}{(M+P)} \\
\end{cases}
$$

For an even number of stages the total cost can be split up in two sums.
The first sum uses the first case and the second sum uses the second case.
$$\text{Total Cost} = \sum_{k=1}^{K/2}\text{cost}_{k} + \sum_{k=K/2+1}^{K}\text{cost}_{k}$$

For an odd number one has to introduce a middle sum.

In [ ]:
display(cost)
arg_min1 = cost.subs({dk:gamma*(k-1)*N/K,dk1:gamma*(k)*N/K})
arg_min2 = cost.subs({dk:gamma*(K+1-k)*M/K,dk1:gamma*(K-k)*M/K})

display(arg_min1)

cost_total_min = transfrom_sum(arg_min1,k,K/2)+transfrom_sum(arg_min2,k,K,n_start=K/2+1)
cost_total_min

In [ ]:
cost_total_min=cost_total_min.subs({P:N,M:N}).expand().collect(K)
cost_total_min

In [ ]:
f = sp.lambdify([K,(N,gamma)],cost_total_min)
Ks = 2**np.arange(11)
Nn = Ks[-1]
for gamma_tilde in np.logspace(-2,0,6):
    costs= f(Ks,(1024,gamma_tilde))
    costs[0] = Nn**2
    plt.semilogy(np.arange(11),costs,label="g="+str(gamma_tilde))
    
plt.legend()
plt.grid()
plt.hlines(Nn**2,0,10)

In [ ]:
f(np.array([2,5,10,25,50]),(100,1))*2 #*2 for anticuasal part

In [ ]:
cost_total_min.subs(K,2)#.subs(gamma,0.39)

### Small test of the coeffs:

Faulhaber coeffs form: https://de.wikipedia.org/wiki/Faulhabersche_Formel 
$$\sum^v_{k=1}k^p$$

In [ ]:
n,N,m=sp.symbols('n N m',postive = True)
transfrom_sum(n**3,n,N,N-1).expand()

In [ ]:
((N-1)**3+N**2).expand()

In [ ]:
n,N,m=sp.symbols('n N m',postive = True)
l = 3
u = 10
for p in range(5):
    print("p=",p)
    s=0
    for k_ in range(l,u+1):
        s += k_**p
    display(s==transfrom_sum(n**p,n,N,n_start=m).subs({N:u,m:l}).simplify())
    #display(transfrom_sum(n**p,n,N,n_start=m).subs({N:u,m:l}).simplify())